In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.3.11'

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///veritabani.db', echo=True)
engine.connect()

2019-11-29 16:03:24,568 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-11-29 16:03:24,571 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:03:24,572 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-11-29 16:03:24,572 INFO sqlalchemy.engine.base.Engine ()


In [3]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [4]:
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    
    def __repr__(self):
        return f"<User(name={self.name}, fullname={self.fullname}, nickname={self.nickname})>"

In [5]:
usr=User(name="Şevket", fullname="Şevket Çakır")

In [6]:
usr

<User(name=Şevket, fullname=Şevket Çakır, nickname=None)>

In [7]:
usr.id

In [8]:
Base.metadata.create_all(engine)

2019-11-29 16:03:24,810 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2019-11-29 16:03:24,811 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:03:24,812 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2019-11-29 16:03:24,812 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:03:24,814 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2019-11-29 16:03:24,815 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:03:24,883 INFO sqlalchemy.engine.base.Engine COMMIT


In [9]:
# Oturum sınıfı oluşturma
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [10]:
session.add(usr)

In [11]:
session.commit()

2019-11-29 16:03:24,965 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-29 16:03:24,967 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-11-29 16:03:24,968 INFO sqlalchemy.engine.base.Engine ('Şevket', 'Şevket Çakır', None)
2019-11-29 16:03:24,970 INFO sqlalchemy.engine.base.Engine COMMIT


In [12]:
usr2 = User(fullname="Cemal Furkan Artan")
usr3 = User(fullname="Merve Arslan")
usr4 = User(fullname="Eda Çeltük")
usr5 = User(fullname="Emre Özdemir")
users = [usr2, usr3, usr4, usr5]
session.add_all(users)
session.commit()

2019-11-29 16:03:25,045 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-29 16:03:25,046 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-11-29 16:03:25,047 INFO sqlalchemy.engine.base.Engine (None, 'Cemal Furkan Artan', None)
2019-11-29 16:03:25,049 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-11-29 16:03:25,050 INFO sqlalchemy.engine.base.Engine (None, 'Merve Arslan', None)
2019-11-29 16:03:25,051 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-11-29 16:03:25,052 INFO sqlalchemy.engine.base.Engine (None, 'Eda Çeltük', None)
2019-11-29 16:03:25,054 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-11-29 16:03:25,054 INFO sqlalchemy.engine.base.Engine (None, 'Emre Özdemir', None)
2019-11-29 16:03:25,055 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
for user in session.query(User).order_by(User.name):
    print(user.name, user.fullname)

2019-11-29 16:03:25,100 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-29 16:03:25,102 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.name
2019-11-29 16:03:25,103 INFO sqlalchemy.engine.base.Engine ()
None Cemal Furkan Artan
None Merve Arslan
None Eda Çeltük
None Emre Özdemir
Şevket Şevket Çakır


In [14]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2019-11-29 16:03:25,139 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2019-11-29 16:03:25,140 INFO sqlalchemy.engine.base.Engine ()
Şevket Şevket Çakır
None Cemal Furkan Artan
None Merve Arslan
None Eda Çeltük
None Emre Özdemir


In [15]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

2019-11-29 16:03:25,177 INFO sqlalchemy.engine.base.Engine SELECT users.name AS name_label 
FROM users
2019-11-29 16:03:25,178 INFO sqlalchemy.engine.base.Engine ()
Şevket
None
None
None
None


In [16]:
#2. ve 3. kaydı alma
for user in session.query(User).order_by(User.name)[1:3]:
    print(user)

2019-11-29 16:03:25,214 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.name
 LIMIT ? OFFSET ?
2019-11-29 16:03:25,215 INFO sqlalchemy.engine.base.Engine (2, 1)
<User(name=None, fullname=Merve Arslan, nickname=None)>
<User(name=None, fullname=Eda Çeltük, nickname=None)>


In [17]:
for user in session.query(User).filter(User.fullname.like('%a%')):
    print(user)

2019-11-29 16:03:25,258 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.fullname LIKE ?
2019-11-29 16:03:25,259 INFO sqlalchemy.engine.base.Engine ('%a%',)
<User(name=Şevket, fullname=Şevket Çakır, nickname=None)>
<User(name=None, fullname=Cemal Furkan Artan, nickname=None)>
<User(name=None, fullname=Merve Arslan, nickname=None)>
<User(name=None, fullname=Eda Çeltük, nickname=None)>


In [18]:
from sqlalchemy.orm import relationship
from sqlalchemy import ForeignKey
class Parent(Base):
    __tablename__ = 'parent'
    id = Column(Integer, primary_key=True)
    children = relationship("Child", back_populates="parent")

class Child(Base):
    __tablename__ = 'child'
    id = Column(Integer, primary_key=True)
    parent_id = Column(Integer, ForeignKey('parent.id'))
    parent = relationship("Parent", back_populates="children")

In [19]:
Base.metadata.create_all(engine)

2019-11-29 16:03:25,347 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2019-11-29 16:03:25,348 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:03:25,349 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("parent")
2019-11-29 16:03:25,350 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:03:25,350 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("parent")
2019-11-29 16:03:25,351 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:03:25,351 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("child")
2019-11-29 16:03:25,352 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:03:25,352 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("child")
2019-11-29 16:03:25,353 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:03:25,354 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE parent (
	id INTEGER NOT NULL, 
	PRIMARY KEY (id)
)


2019-11-29 16:03:25,355 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:03:25,395 INFO sqlalchemy.engi

In [20]:
p=Parent()
c=Child()

In [21]:
p.children=[c]

In [22]:
session.add(p)
session.commit()

2019-11-29 16:03:25,540 INFO sqlalchemy.engine.base.Engine INSERT INTO parent DEFAULT VALUES
2019-11-29 16:03:25,541 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:03:25,543 INFO sqlalchemy.engine.base.Engine INSERT INTO child (parent_id) VALUES (?)
2019-11-29 16:03:25,544 INFO sqlalchemy.engine.base.Engine (1,)
2019-11-29 16:03:25,545 INFO sqlalchemy.engine.base.Engine COMMIT


In [23]:
p.children

In [24]:
c.parent

In [25]:
from sqlalchemy import DateTime
import datetime
#Ogrenci sınıfı tanımı
class Ogrenci(Base):
    __tablename__ = "ogrenciler"
    
    no = Column(Integer, primary_key=True)
    adi = Column(String)
    soyadi = Column(String)
    grup_id = Column(Integer, ForeignKey("gruplar.id"))
    grup = relationship("Grup", back_populates='ogrenciler')
    def __repr__(self):
        return f"<Ogrenci({self.no} - {self.adi} {self.soyadi})>"

class Grup(Base):
    __tablename__ = "gruplar"
    
    id = Column(Integer, primary_key=True)
    konu = Column(String)
    created = Column(DateTime, default=datetime.datetime.utcnow)
    ogrenciler = relationship("Ogrenci", back_populates='grup')

Base.metadata.create_all(engine)

2019-11-29 16:17:28,687 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2019-11-29 16:17:28,688 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:17:28,689 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("parent")
2019-11-29 16:17:28,690 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:17:28,691 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("child")
2019-11-29 16:17:28,692 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:17:28,693 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("ogrenciler")
2019-11-29 16:17:28,693 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:17:28,694 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("ogrenciler")
2019-11-29 16:17:28,695 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:17:28,695 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("gruplar")
2019-11-29 16:17:28,696 INFO sqlalchemy.engine.base.Engine ()
2019-11-29 16:17:28,697 INFO sqlalchemy.engine.base.Engine PRAGMA tem

In [27]:
gruplar=[]
gruplar.append(Grup(konu="Webscrap ile içerik çekme: Premier Lig"))
gruplar.append(Grup(konu="OpenCV Görüntü İşleme"))
gruplar.append(Grup(konu="Pygame 2D Oyun"))
gruplar[0].ogrenciler = [Ogrenci(no=15253003, adi="Cemal Furkan", soyadi="Artan"), Ogrenci(no=15253072,adi="Mustafa", soyadi="Zeycan")]
gruplar[1].ogrenciler = [Ogrenci(no=15253016, adi="Eda", soyadi="Çeltük"), Ogrenci(no=14253006,adi="Merve", soyadi="Arslan")]
gruplar[2].ogrenciler = [Ogrenci(no=16253055, adi="Emre", soyadi="Özdemir")]
session.add_all(gruplar)
session.commit()

2019-11-29 16:24:22,314 INFO sqlalchemy.engine.base.Engine INSERT INTO gruplar (konu, created) VALUES (?, ?)
2019-11-29 16:24:22,315 INFO sqlalchemy.engine.base.Engine ('Webscrap ile içerik çekme: Premier Lig', '2019-11-29 13:24:22.314788')
2019-11-29 16:24:22,317 INFO sqlalchemy.engine.base.Engine INSERT INTO gruplar (konu, created) VALUES (?, ?)
2019-11-29 16:24:22,318 INFO sqlalchemy.engine.base.Engine ('OpenCV Görüntü İşleme', '2019-11-29 13:24:22.317788')
2019-11-29 16:24:22,319 INFO sqlalchemy.engine.base.Engine INSERT INTO gruplar (konu, created) VALUES (?, ?)
2019-11-29 16:24:22,319 INFO sqlalchemy.engine.base.Engine ('Pygame 2D Oyun', '2019-11-29 13:24:22.319788')
2019-11-29 16:24:22,321 INFO sqlalchemy.engine.base.Engine INSERT INTO ogrenciler (no, adi, soyadi, grup_id) VALUES (?, ?, ?, ?)
2019-11-29 16:24:22,322 INFO sqlalchemy.engine.base.Engine ((15253003, 'Cemal Furkan', 'Artan', 1), (15253072, 'Mustafa', 'Zeycan', 1), (15253016, 'Eda', 'Çeltük', 2), (14253006, 'Merve', '